# Bifurcation diagram export

$\textbf{Description:}$ Exports two csv files for a birfucation diagram with $A-q$ axis and $A-p$ axis.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import *
from scipy.integrate import odeint
import csv
from ipywidgets import FloatProgress
from IPython.display import display
from numba import jit
import time

In [2]:
#function for integrator 
@jit
def f(r,t,parameters):
    #Parameters
    b = parameters[0]
    A = parameters[1]
    om = parameters[2]
    
    x = r[0]
    y = r[1]
    drdt = np.array([ y, - 2*b*y + x - x**3 + A*cos(om*t)])
    return drdt

# integrator 
def Duff(r0,T, parameters):
    # no. data points per cycle
    dens = 500
    def tempf(r,t):
        return f(r,t,parameters)
    tvals = np.linspace(0,2*pi*T,dens*T)
    temp = odeint(tempf,r0,tvals)
    return temp

# for given parameters, chooses a random point evolves 100 periods and samples 100 poincare return maps. Repeats this 9 times.
@jit
def traj(parameters):    
    para = np.array(parameters)
    
    temp = [[np.random.uniform(-1.5,1.5),np.random.uniform(-1,1)]]
    for i in range(0,100,1):
        temp = np.append(temp,[Duff(temp[-1],1,para)[-1]],axis=0)
    traj = [temp[-1]]
    for i in range(0,100,1):
        traj = np.append(traj,[Duff(traj[-1],1,para)[-1]],axis=0)
        
    for j in range(0,4,1):
        temp = np.append(temp,[[np.random.uniform(-1.5,0),np.random.uniform(-1,1)]],axis = 0)
        for i in range(0,100,1):
            temp = np.append(temp,[Duff(temp[-1],1,para)[-1]],axis=0)
        traj = np.append(traj,[temp[-1]],axis=0)
        for i in range(0,100,1):
            traj = np.append(traj,[Duff(traj[-1],1,para)[-1]],axis=0)
    
    for j in range(0,4,1):
        temp = np.append(temp,[[np.random.uniform(0,1.5),np.random.uniform(-1,1)]],axis = 0)
        for i in range(0,100,1):
            temp = np.append(temp,[Duff(temp[-1],1,para)[-1]],axis=0)
        traj = np.append(traj,[temp[-1]],axis=0)
        for i in range(0,100,1):
            traj = np.append(traj,[Duff(traj[-1],1,para)[-1]],axis=0)    
    
    return traj

#exports csv files of bifurcation diagrams in x and y with fixed om = 1 and b = 0.15 fixed. 
#F varies from first to last with num F values sampled. 
#filenamex and filenamey must be in quotation marks and have .csv at the end

def bifurcation_diagram(first,last, num,filenamex,filenamey):
    # prepares csv writer
    csvfilex = open(filenamex, 'a', newline='') 
    writerx = csv.writer(csvfilex, delimiter=',')
    csvfiley = open(filenamey, 'a', newline='') 
    writery = csv.writer(csvfiley, delimiter=',')

    st = time.time() #timer
    lbar = FloatProgress(min=0, max=num) #loading bar
    display(lbar)
    #main loop
    for m in range(0,num,1):
        lbar.value = m
        # generates slice at certian F value
        para = [0.15, first + m*(last-first)/num,1]
        temp = traj(para)
        # appends x values of slice with value of F (i.e. [[f,x1],...]) to file. similarly for y.
        tmpx = [[0,0]]
        tmpy = [[0,0]]
        for i in range(0,len(temp),1):
            tmpx = np.append(tmpx,np.array([[ first + m*(last-first)/num, temp[i][0]]]),axis = 0)
            tmpx = np.delete(tmpx,0,0)
            tmpy = np.append(tmpy,np.array([[ first + m*(last-first)/num, temp[i][1]]]),axis = 0)
            tmpy = np.delete(tmpy,0,0)
            writerx.writerows(tmpx)
            writery.writerows(tmpy)

    end = time.time()
    print(end - st)

    
    

In [3]:
bifurcation_diagram(0.205,0.226, 10000,'bifurcx_0p205to0p226_10000.csv','bifurcy_0p205to0p226_10000.csv')

11028.909650802612


In [5]:
bifurcation_diagram(0.217,0.21925, 10000,'bifurcx_0p217to0p21925_10000.csv','bifurcy_0p217to0p21925_10000.csv')

10057.25465297699
